Week 5, Assignment C1:

Logistic Regression with Yahoo! Finance API in Python

In this notebook, we will study Logistic Regression using the Yahoo! Finance API in Python.

Please complete the lines bellow where you see "#TODO".

In [1]:
!pip install yfinance

In [2]:
!pip install pandas_datareader

In [3]:
#Objective 1.
#Use this cell to import the Numpy (as np), Pandas (as pd), and YFinance (as yf) packages.

import numpy as np
import pandas as pd
import yfinance as yf 

#Our work will also require some components of the Sklearn and Pandas_Datareader packages as imported below:

from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from pandas_datareader import data as pdr
yf.pdr_override()

In [4]:
#Objective 2.
#Select a stock symbol for a stock whose historical data is available on the Yahoo! Finance website.  Store the
#string of your chosen symbol to the new variable "stock_symbol".

stock_symbol = "UBER"

In [5]:
#Objective 3.
#Use the function "pdr.get_data_yahoo(stock_symbol, start_date, end_date)" to generate a Pandas dataframe of
#historical stock data for your chosen stock.  Retain only the first four columns of the dataframe.  Be sure to 
#drop any rows containing NaN's, and take a peek at the resulting dataframe to make sure everything looks good.

df = pdr.get_data_yahoo(stock_symbol, pd.Timestamp('2019'), pd.Timestamp('2021'))
df = df[df.columns[0:4]]
df = df[['Open', 'High', 'Low', 'Close']]
df = df.dropna()

[*********************100%***********************]  1 of 1 completed


In [6]:
df

,Open,High,Low,Close
Date,,,,
2019-05-10,42.000000,45.000000,41.060001,41.570000
2019-05-13,38.790001,39.240002,36.080002,37.099998
2019-05-14,38.310001,39.959999,36.849998,39.959999
2019-05-15,39.369999,41.880001,38.950001,41.290001
2019-05-16,41.480000,44.060001,41.250000,43.000000
...,...,...,...,...
2020-12-24,53.599998,53.625000,52.290001,52.880001
2020-12-28,53.020000,53.130001,51.560001,51.970001
2020-12-29,52.330002,52.990002,51.470001,52.270000


In [7]:
#Objective 4.
#Use the historical data to define some predictor variables.  Add these variables to the dataframe.  Include, at a
#minimum, (Predictor.I) the rolling average closing price over the past fifteen (15) days and (Predictor.II) the
#change in opening price over the past one (1) day.  Be sure to again drop any rows containing NaN's, and take a
#peek at the resulting dataframe to make sure everything looks good.  Store the dataframe as the new variable "X".

df['Close_15_Rolling'] = df['Close'].rolling(window=15).mean()
df['Open_1_Change'] = df['Open'].diff()
df = df.dropna()
X = df

In [8]:
#Objective 5.
#Define the target or dependent variable to be one (1) if the change in closing price over the past one (1) day is
#nonnegative and negative one (-1) if the change in closing price over the past one (1) day is negative.  This
#variable's values should be forward-looking (i.e., you should subtract today's price from tomorrow's price rather
#than subtracting yesterday's price from today's price).  Store the resulting values as the new variable "y".  You
#may find the "np.where(*args)" function to be useful.  Look it up in Numpy documentation for support.

y = np.where((df["Close"].shift(-1) - df["Close"]) > 0, 1, -1) 


In [9]:
#Objective 6.
#Split the data into training and test sets, putting the first seventy percent (70%) of the data in the training
#set.

index = int(0.7*len(X))
X_train = X[:index]
X_test  = X[index:]
y_train = y[:index]
y_test  = y[index:]

In [10]:
#Objective 7.
#Instantiate the Logistic Regression model object, and use its ".fit(*args)" method to fit the model to the 
#training data.
logistic = LogisticRegression()
logistic = logistic.fit(X_train, y_train)

In [18]:
#Objective 8.
#Examine the model's coefficients by using its ".coef_" method.

pd.DataFrame(zip(X.columns, np.transpose(logistic.coef_)))
#pd.DataFrame({'column' :X.columns.values, 'b': logistic.coef_})

,0,1
0,Open,[-0.20816519669589947]
1,High,[0.35335994214610383]
2,Low,[0.3317385358745953]
3,Close,[-0.5013982423362009]
4,Close_15_Rolling,[-0.03524256406027356]
5,Open_1_Change,[0.0765885066483232]


In [12]:
#Objective 9.
#Use the model's ".predict_proba(*args)" and ".predict(*args)" methods to generate predictions over the test set.

probabilities = logistic.predict_proba(X_test)
predictions = logistic.predict(X_test)

In [13]:
#Objective 10.
#Use the function "metrics.confusion_matrix(*args)" to create a confusion matrix comparing the predicted and true
#classification labels over the test set.

metrics.confusion_matrix(y_test, predictions)

array([[44, 17],
       [48, 12]])

In [14]:
#Objective 11.
#Calculate the model's accuracy on the test set using its ".score(*args)" method.

logistic.score(X_test,y_test)

0.4628099173553719

In [15]:
#Objective 12.
#Use five-fold cross validation to cross-check the accuracy of the model over different held-out test sets.  This
#is where you should use the function "cross_val_score(*args)".

cross_val = cross_val_score(logistic, X_test, y_test, cv=5)

In [16]:
cross_val

array([0.56      , 0.58333333, 0.58333333, 0.54166667, 0.5       ])